### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
input_song = input("Enter your desired song name here!")
input_playlist = input("Enter your desired playlist name here!s")
recc_num = int(input("Enter how many songs you want to be recommended!"))
client_id = input("Enter your Spotify API Client Key here!")
client_secret = input("Enter your Spotify API Secret Key here!")

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [6]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": track["artists"][0]["name"],
                      "id_artists": track["artists"][0]["id"], "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: 22 (Taylor's Version) by Taylor Swift from the album Red (Taylor's Version).


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,Taylor Swift,06HL4z0CvFAxyc27GXpf02,2021-11-12


In [7]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [8]:
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,Taylor Swift,06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,-5.62,1,0.0281,0.000443,0.00001,0.0753,0.642,103.984,4,2021


In [9]:
import ast

td['artists'] = "['" + td['artists'] + "']"
td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,-5.62,1,0.0281,0.000443,0.00001,0.0753,0.642,103.984,4,2021


In [10]:
td['artists_count'] = len(td['artists'])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,1,0.0281,0.000443,0.00001,0.0753,0.642,103.984,4,2021,1


In [11]:
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,0.0281,0.000443,0.00001,0.0753,0.642,103.984,4,2021,1,Taylor Swift


In [12]:
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(len(td['artists_count'])):
    td['genres'] = td['artist_' + str(i+1)].apply(get_artist_genres)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,0.000443,0.00001,0.0753,0.642,103.984,4,2021,1,Taylor Swift,[pop]


### Adding Input to Dataset and further processing

In [13]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,tempo,time_signature,year,artists_count,artist_1,genres,song_name_artist,artist_2,artist_3,artist_4
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,103.984,4,2021,1,Taylor Swift,[pop],NaN,NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,113.564,3,2008,1,Gerry & The Pacemakers,"[folk rock, british invasion, merseybeat, rock...",You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,120.689,4,2020,1,The Toys,[],A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,120.689,4,2020,1,The Toys,[thai pop],A Lover's Concerto['The Toys'],NaN,NaN,NaN
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,104.536,4,2008,1,Gerry & The Pacemakers,"[folk rock, british invasion, merseybeat, rock...",Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN


In [14]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(228030, 29)
(186012, 29)


In [15]:
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,song_name_artist,artist_2,artist_3,artist_4,song_id_name,genres_count
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),79,230960,0,[Taylor Swift],06HL4z0CvFAxyc27GXpf02,2021-11-12,0.642,0.695,...,2021,1,Taylor Swift,[pop],NaN,NaN,NaN,NaN,3yII7UwgLF6K5zW3xad3MP22 (Taylor's Version),1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[folk rock, british invasion, merseybeat, rock...",You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],A Lover's Concerto['The Toys'],NaN,NaN,NaN,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[folk rock, british invasion, merseybeat, rock...",Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[folk rock, british invasion, merseybeat, rock...",Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [16]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186012, 30)
(166252, 30)


### Supersetting genres

In [17]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'pop': 6881, 'folk rock': 933, 'british invasion': 495, 'merseybeat': 529, 'rock-and-roll': 650, 'rockabilly': 550, 'classic uk pop': 704, 'adult standards': 1216, 'bubblegum pop': 416, 'brill building pop': 763, 'lounge': 283, 'easy listening': 126, 'canadian blues': 54, 'british blues': 265, 'mellow gold': 1141, 'lilith': 307, 'singer-songwriter': 312, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'harlem renaissance': 23, 'big band': 94, 'stride': 75, 'cool jazz': 274, 'jazz': 304, 'swing': 56, 'jazz piano': 73, 'vocal jazz': 355, 'contemporary jazz': 73, 'jazz saxophone': 71, 'swedish jazz': 134, 'jazz quartet': 20, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'jazz fusion': 297, 'free jazz': 58, 'classic soundtrack': 89, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'soundtrack': 456, 'hard bop': 204, 'jazz trumpet': 176, 'psychedelic rock': 592, 'roots rock': 250, 'acid rock': 67, 'classic rock': 1358, 'hard rock': 728, '

In [18]:
import matplotlib.pyplot as plt

# # Plot the frequency distribution
# plt.bar(genre_dict.keys(), genre_dict.values())
# plt.xlabel('Genres')
# plt.ylabel('Frequency')
# plt.title('Frequency of Various Genres')
# plt.show()

In [19]:
genres_df = pd.Series(genre_dict)
genres_df.head()

pop                 6881
folk rock            933
british invasion     495
merseybeat           529
rock-and-roll        650
dtype: int64

In [20]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4229.000000
mean      130.132419
std       393.314261
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6881.000000
dtype: float64

In [21]:
# genres_df.to_csv("data/genre_frequency.csv")

In [22]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


pop: ['pop', 'classic uk pop', 'bubblegum pop', 'brill building pop', 'sunshine pop', 'baroque pop', 'dance pop', 'pop rock', 'post-teen pop', 'pop rap', 'new wave pop', 'finnish pop', 'finnish dance pop', 'uk pop punk', 'europop', 'canadian pop', 'barbadian pop', 'latin pop', 'acoustic pop', 'spanish pop rock', 'mexican pop', 'latin arena pop', 'spanish pop', 'bow pop', 'pop nacional', 'socal pop punk', 'britpop', 'sophisti-pop', 'hip pop', 'south african pop', 'south african pop dance', 'bahamian pop', 'puerto rican pop', 'canadian pop punk', 'power-pop punk', 'deep power-pop punk', 'antiviral pop', 'j-pop girl group', 'australian pop', 'art pop', 'indie pop', 'pop punk', 'synthpop', 'norwegian pop', 'candy pop', 'ambient pop', 'glitch pop', 'french pop', 'dream pop', 'italian adult pop', 'indonesian pop', 'electropop', 'german pop rock', 'german pop', 'tatar pop', 'collage pop', 'pop r&b', 'belarusian pop', 'pop rock brasileiro', 'c-pop', 'cantopop', 'mandopop', 'classic mandopop', 

In [23]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

genres_df = pd.DataFrame(genres_df, columns=['genres'])

def group_into_supersets(new_df, superset_mapping):
    def get_superset(genres):
        for superset, subsets in superset_mapping.items():
            if any(subset in genre for genre in genres for subset in subsets):
                return superset
        return 'other'  # If no superset is found

    new_df['supersetted_genres'] = new_df['genres'].apply(get_superset)
    return new_df

# Group into supersets and create new column
new_df = group_into_supersets(new_df, superset_mapping)

In [24]:
new_df['supersetted_genres'].value_counts().count()

833

### Encoding

In [25]:
# One-hot Encoding of all the superset genres of each song in the dataframe.

genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack()).groupby(level=0).sum()

# Concatenate the binary columns with the original DataFrame
df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [26]:
df_2.iloc[:,0:20].describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,166252.000000,1.662520e+05,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000,166252.000000
mean,40.123217,2.328364e+05,0.107283,0.606616,0.653451,5.330071,-7.320532,0.609123,0.091713,0.299998,0.063974,0.207366,0.535137,121.775761,3.929324
std,17.117229,1.013132e+05,0.309474,0.157524,0.216637,3.565033,3.642153,0.487948,0.112539,0.292923,0.206059,0.183005,0.249318,29.153060,0.361895
min,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,1.911600e+05,0.000000,0.504000,0.511000,2.000000,-8.686000,0.000000,0.034200,0.039200,0.000000,0.096900,0.335000,98.006000,4.000000
50%,41.000000,2.237130e+05,0.000000,0.617000,0.681000,5.000000,-6.603000,1.000000,0.047800,0.199000,0.000002,0.131000,0.536000,121.927000,4.000000
75%,52.000000,2.624270e+05,0.000000,0.721000,0.829000,9.000000,-5.028000,1.000000,0.092800,0.514000,0.000489,0.265000,0.741000,139.952000,4.000000
max,100.000000,4.995083e+06,1.000000,0.991000,1.000000,11.000000,2.854000,1.000000,0.966000,0.996000,1.000000,1.000000,1.000000,220.099000,5.000000


In [27]:
# Removing unnecessary features.

df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'mode', 'danceability', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'supersetted_genres', 'explicit', 'time_signature'], axis=1)
df_2.head()

,id,name,duration_ms,artists,energy,key,loudness,speechiness,acousticness,instrumentalness,...,yoga,zen,zespol dzieciecy,zhongguo feng,zikir,zillertal,zim urban groove,zolo,zouglou,zouk
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),230960,[Taylor Swift],0.695,7,-5.620,0.0281,0.000443,0.000010,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,NaN,0.265,0,-11.101,0.0322,0.394000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,NaN,0.365,6,-10.226,0.0289,0.255000,0.000005,...,0,0,0,0,0,0,0,0,0,0
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,NaN,0.352,1,-14.165,0.0300,0.406000,0.000000,...,0,0,0,0,0,0,0,0,0,0
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,NaN,0.201,7,-17.796,0.0623,0.887000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,energy,key,loudness,speechiness,acousticness,instrumentalness,...,yoga,zen,zespol dzieciecy,zhongguo feng,zikir,zillertal,zim urban groove,zolo,zouglou,zouk
0,3yII7UwgLF6K5zW3xad3MP,22 (Taylor's Version),230960,[Taylor Swift],0.695,7,-5.620,0.0281,0.000443,0.000010,...,0,0,0,0,0,0,0,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.265,0,-11.101,0.0322,0.394000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,0,0.365,6,-10.226,0.0289,0.255000,0.000005,...,0,0,0,0,0,0,0,0,0,0
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.352,1,-14.165,0.0300,0.406000,0.000000,...,0,0,0,0,0,0,0,0,0,0
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.201,7,-17.796,0.0623,0.887000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_2.shape

(166252, 849)

### Cosine Similarity & Output Extraction

In [30]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Normalize numerical features
    numerical_features = ['loudness', 'tempo']
    scaler = MinMaxScaler()
    df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])
    
    # Standardize Year
    df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])
    df_2 = df_2.drop(['year'], axis=1)

    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.8294518182206949,
 0.9230349173796657,
 0.8673810504088575,
 0.5547462651591607,
 0.5381957008062338,
 0.9142757797798362,
 0.9101871433352313,
 0.8043645763746912,
 0.8890640420728219,
 0.910402600656305,
 0.8540837616523098,
 0.8191378956907134,
 0.8975994950197491,
 0.4671795857803906,
 0.614958164480069,
 0.5461435605415081,
 0.6124090455622805,
 0.644999266765939,
 0.5649608323212949,
 0.6029893116402696,
 0.585350042303169,
 0.9315833521655026,
 0.9172739540883547,
 0.5960800118501255,
 0.8723908958048691,
 0.624796717169392,
 0.5323377200507685,
 0.45956809758675027,
 0.5248323536939706,
 0.45214838468521656,
 0.5507012198025775,
 0.6654808200186977,
 0.8667391851943789,
 0.5961060555153967,
 0.9202246123078412,
 0.6551335624060919,
 0.6882869067694113,
 0.825541150671555,
 0.8721954946994788,
 0.9012321276757227,
 0.9319713243001115,
 0.8699037224890194,
 0.7416962776982173,
 0.6690821779699652,
 0.7105665749820378,
 0.8747576346311803,
 0.8747827036789405,
 0.89876298497565

In [31]:
max(arr[1:])

0.9987216134322578

In [32]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.9:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[1, 5, 6, 9, 21, 22, 34, 39, 40, 51, 54, 55, 57, 63, 64, 65, 68, 71, 73, 74, 75, 76, 80, 81, 83, 84, 85, 86, 87, 89, 99, 104, 106, 108, 111, 118, 124, 133, 135, 137, 138, 139, 142, 144, 274, 275, 289, 292, 298, 300, 301, 303, 308, 315, 326, 403, 421, 428, 433, 435, 1079, 1087, 1090, 1096, 1100, 1102, 1104, 1107, 1115, 1126, 1130, 1134, 1163, 1165, 1187, 1189, 1191, 1248, 1250, 1252, 1254, 1256, 1622, 1634, 1761, 1797, 1896, 1940, 1954, 2065, 2068, 2084, 2108, 2110, 2128, 2130, 2135, 2144, 2227, 2236, 2238, 2240, 2242, 2246, 2251, 2256, 2258, 2266, 2271, 2279, 2287, 2288, 2295, 2299, 2305, 2314, 2319, 2323, 2326, 2328, 2339, 2340, 2346, 2355, 2360, 2372, 2374, 2390, 2395, 2399, 2401, 2404, 2405, 2435, 2441, 2443, 2447, 2455, 2459, 2461, 2463, 2465, 2467, 2480, 2482, 2490, 2496, 2501, 2503, 2504, 2515, 2519, 2529, 2576, 2579, 2589, 2591, 2594, 2613, 2625, 2642, 2644, 2648, 2650, 2652, 2659, 2661, 2681, 2693, 2695, 2703, 2708, 2742, 2744, 2760, 2768, 2770, 2809, 2812, 2813, 2817, 2823, 28

In [33]:
len(suggestions)

54605

In [34]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,energy,key,loudness,speechiness,acousticness,instrumentalness,...,zen,zespol dzieciecy,zhongguo feng,zikir,zillertal,zim urban groove,zolo,zouglou,zouk,standardized_year
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.265,0,0.777978,0.0322,0.394,0.000000,...,0,0,0,0,0,0,0,0,0,0.380952
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.352,1,0.729230,0.0300,0.406,0.000000,...,0,0,0,0,0,0,0,0,0,0.380952
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.201,7,0.671461,0.0623,0.887,0.000000,...,0,0,0,0,0,0,0,0,0,0.857143
9,25qyOLQyX7bsceW3UL55BW,Hello Little Girl,115240,0,0.639,6,0.833233,0.0302,0.288,0.000000,...,0,0,0,0,0,0,0,0,0,0.380952
21,0GohHD8bn4lP83agkBvi5i,"Eastern Rain - Live at The 2nd Fret, Philadelp...",233520,0,0.212,6,0.714704,0.0406,0.932,0.000023,...,0,0,0,0,0,0,0,0,0,0.952381


In [35]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

39533

In [36]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,...,zespol dzieciecy,zhongguo feng,zikir,zillertal,zim urban groove,zolo,zouglou,zouk,standardized_year,artists
5,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0.352,1,0.729230,0.0300,0.406,0.000000,0.122,...,0,0,0,0,0,0,0,0,0.380952,['Gerry & The Pacemakers']
6,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0.201,7,0.671461,0.0623,0.887,0.000000,0.904,...,0,0,0,0,0,0,0,0,0.857143,['Frank Sinatra']
9,25qyOLQyX7bsceW3UL55BW,Hello Little Girl,115240,0.639,6,0.833233,0.0302,0.288,0.000000,0.343,...,0,0,0,0,0,0,0,0,0.380952,['Gerry & The Pacemakers']
21,0GohHD8bn4lP83agkBvi5i,"Eastern Rain - Live at The 2nd Fret, Philadelp...",233520,0.212,6,0.714704,0.0406,0.932,0.000023,0.692,...,0,0,0,0,0,0,0,0,0.952381,['Joni Mitchell']
22,4JyPPRoW8y6mA3XA7gKvoL,Intro To The Circle Game - Live at The 2nd Fre...,232640,0.153,10,0.610844,0.3780,0.911,0.000000,0.358,...,0,0,0,0,0,0,0,0,0.952381,['Joni Mitchell']


### Recommendations Output

In [37]:
# Generating recc_num of the matched songs at random.
def recommend(df_main, recc_num):
    recs = []
    if len(df_main) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_main) < recc_num:
        recc_num = len(df_main)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_main.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_main = df_main.drop(list(recc['id'])[0])
        recs.append((name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_main, recc_num) # End Product

Here are your recommendations!
1. Dance by 'Jurga' published in 2018
2. Só Tu Podes Alcançar by '4Taste' published in 2006
3. Incondicional - Ao Vivo by 'Luan Santana' published in 2012
4. Insomnia by 'Daya' published in 2019
5. I Am Moana (Song of the Ancestors) by 'Rachel House', "Auli'i Cravalho" published in 2016
6. Die Onverkrygbare by 'Chris Chameleon' published in 2006
7. Psalms No. 122 by 'David and the High Spirit' published in 2010
8. Hasret Türküsü by 'Suavi' published in 2011
9. 憑著愛 by 'Julie Sue' published in 2015
10. Resposta - Ao Vivo by 'Skank' published in 2001


In [38]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.search(q=f"track:{song[0]} year:{song[1]}", type="track")
    print(result)
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ADance+year%3A2018&type=track&offset=0&limit=10', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7n2Ycct7Beij7Dj7meI4X0'}, 'href': 'https://api.spotify.com/v1/artists/7n2Ycct7Beij7Dj7meI4X0', 'id': '7n2Ycct7Beij7Dj7meI4X0', 'name': 'TWICE', 'type': 'artist', 'uri': 'spotify:artist:7n2Ycct7Beij7Dj7meI4X0'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', '

{'snapshot_id': 'Miw3NTY3YTk1MDNmYTBkYWE1Mjc2OWRkNDgxMjdlYjI1YTJjMWY3YWM1'}